In [1]:
import os
import glob
import lsst.eotest.sensor as sensorTest
import lsst.eotest.image_utils as imutils
import bot_eo_analyses as bot_eo
from camera_components import camera_info
from lsst.eotest.sensor.flatPairTask import find_flat2    # for use with TS8 data in the flat_pairs and ptc tasks

In [2]:
# Specify a raft and a run to analyze.
root_dir = '/gpfs/slac/lsst/fs1/g/data/jobHarness/jh_archive/LCA-11021_RTM'
raft = 'LCA-11021_RTM-014'
run = '9537'

# Pick a CCD by slot.
slot = 'S00'
det_name = '{}_{}'.format(raft, slot)

In [3]:
# File to gather the eotest results.  A file with this name is updated by the various eotasks.
eotest_results_file = '_'.join((det_name, run, 'eotest_results.fits'))

# A function to help gather the data.
def glob_files(*args):
    return sorted(glob.glob(os.path.join(root_dir, raft, run, *args)))

# For the gain/psf and read noise, we use the Fe55 acquisition.
job = 'fe55_raft_acq'
bias_files = glob_files('fe55_raft_acq', 'v0', '*', slot, '*fe55_bias*.fits')
fe55_files = glob_files('fe55_raft_acq', 'v0', '*', slot, '*fe55_fe55*.fits')

# For full raft noise correlation matrix, one needs to crate a dictionary of bias files, keyed by slot.
bias_file_dict = {slot: glob_files('fe55_raft_acq', 'v0', '*', slot, '*fe55_bias*.fits')[0]
                  for slot in camera_info.get_slot_names()}

# For bright defects and dark current tasks.
dark_files = glob_files('dark_raft_acq', 'v0', '*', slot, '*dark_dark*.fits')

# For dark defects and CTE tasks.
sflat_files = glob_files('sflat_raft_acq', 'v0', '*', slot, '*sflat_500_flat_H*.fits')

# For linearity and ptc tasks.
flat1_files = glob_files('flat_pair_raft_acq', 'v0', '*', slot, '*flat1*.fits')

# Set default gain values in case the Fe55 task is skipped.
gains = {_: 0.8 for _ in range(1, 17)}

In [4]:
# Run the Fe55 single sensor task.
%time bot_eo.fe55_task(run, det_name, fe55_files, bias_files)

# Retrieve the calculated gains from the eotest results file. 
#eotest_results = sensorTest.EOTestResults(eotest_results_file)
#gains = {amp: gain for amp, gain in zip(eotest_results['AMP'], eotest_results['GAIN'])}

/opt/lsst/software/stack/python/miniconda3-4.3.21/lib/python3.6/site-packages/scipy/optimize/minpack.py:436: RuntimeWarning: Number of calls to function has reached maxfev = 1000.
  warnings.warn(errors[info][0], RuntimeWarning)
/gpfs/slac/lsst/fs3/g/data/software/jh_install/lssttd-1369/eotest-0.1.0/python/lsst/eotest/sensor/fe55_psf.py:57: RuntimeWarning: invalid value encountered in sqrt
  std_y = np.sqrt((sum_y2 / sum_0) - (mean_y * mean_y))
/gpfs/slac/lsst/fs3/g/data/software/jh_install/lssttd-1369/eotest-0.1.0/python/lsst/eotest/sensor/fe55_psf.py:404: RuntimeWarning: invalid value encountered in greater
  indx = np.where((chiprob > min_prob) & (amps == amp))
/gpfs/slac/lsst/fs3/g/data/software/jh_install/lssttd-1369/eotest-0.1.0/python/lsst/eotest/sensor/fe55_psf.py:56: RuntimeWarning: invalid value encountered in sqrt
  std_x = np.sqrt((sum_x2 / sum_0) - (mean_x * mean_x))
/gpfs/slac/lsst/fs3/g/data/software/jh_install/lssttd-1369/eotest-0.1.0/python/lsst/eotest/sensor/EOTestPlo

CPU times: user 1min 13s, sys: 18.5 s, total: 1min 32s
Wall time: 1min 21s


In [5]:
# Run the read noise task.
%time bot_eo.read_noise_task(run, det_name, bias_files, gains)

CPU times: user 16.5 s, sys: 3.36 s, total: 19.9 s
Wall time: 17.9 s


In [6]:
# Run the raft-level overscan correlation calculation.
%time bot_eo.raft_noise_correlations(run, raft, bias_file_dict)

CPU times: user 27.5 s, sys: 2.02 s, total: 29.5 s
Wall time: 32.6 s


In [7]:
# Run the bright defects task.
mask_files = sorted(glob.glob('_'.join((det_name, run, '*mask.fits'))))
%time bot_eo.bright_defects_task(run, det_name, dark_files, gains=gains, mask_files=mask_files)

24396.0
23223.0
22399.0
22360.0
22639.0
23115.0
24607.0
25856.0
27939.0
25000.0
23691.0
22635.0
21632.0
22084.0
22769.0
24570.0
CPU times: user 31.4 s, sys: 2.17 s, total: 33.6 s
Wall time: 36.2 s


In [8]:
# Run the dark defects task.
mask_files = sorted(glob.glob('_'.join((det_name, run, '*mask.fits'))))
%time bot_eo.dark_defects_task(run, det_name, sflat_files, mask_files=mask_files)

50450.0
50710.0
50571.0
50309.0
50138.0
50744.0
51157.0
51134.0
50007.0
49728.0
49183.0
48661.0
48627.0
48991.0
49285.0
49372.0
CPU times: user 48 s, sys: 3.37 s, total: 51.4 s
Wall time: 57.1 s


In [9]:
# Run the dark current task.
mask_files = sorted(glob.glob('_'.join((det_name, run, '*mask.fits'))))
%time dark_curr_pixels, dark95s = bot_eo.dark_current_task(run, det_name, dark_files, gains, mask_files=mask_files)
bot_eo.plot_ccd_total_noise(run, det_name, dark_curr_pixels, dark95s, eotest_results_file)

24396.0
23223.0
22399.0
22360.0
22639.0
23115.0
24607.0
25856.0
27939.0
25000.0
23691.0
22635.0
21632.0
22084.0
22769.0
24570.0
CPU times: user 59.1 s, sys: 1.09 s, total: 1min
Wall time: 1min


In [10]:
# Run the CTE task.
mask_files = sorted(glob.glob('_'.join((det_name, run, '*mask.fits'))))
%time superflat_file = bot_eo.cte_task(run, det_name, sflat_files, gains, mask_files=mask_files)
bot_eo.plot_cte_results(run, det_name, superflat_file, eotest_results_file, mask_files=mask_files);

74828.0
73908.0
72952.0
72651.0
72760.0
73842.0
75758.0
76986.0
77940.0
74718.0
72831.0
71280.0
70236.0
71052.0
72031.0
73936.0
CPU times: user 27.5 s, sys: 663 ms, total: 28.2 s
Wall time: 28.3 s


In [11]:
# Run the linearity task
mask_files = sorted(glob.glob('_'.join((det_name, run, '*mask.fits'))))
flat1_files_subset = flat1_files[::len(flat1_files)//4]   # downselect so that this demo runs more quickly
%time bot_eo.flat_pairs_task(run, det_name, flat1_files_subset, gains, mask_files=mask_files, flat2_finder=find_flat2)

amp, flux bounds, fit_pars: 1 -491.22163350694314 3109.1620367187643 [   22.21985414 20914.87304687]
amp, flux bounds, fit_pars: 2 -493.4845996365862 3088.7953415519764 [   22.33214637 21020.5703125 ]
amp, flux bounds, fit_pars: 3 -492.0352414032569 3101.839565651941 [   22.2600965  20952.75195312]
amp, flux bounds, fit_pars: 4 -489.3183882980305 3126.291243598979 [   22.12628247 20826.796875  ]
amp, flux bounds, fit_pars: 5 -487.29931785513656 3144.462877585023 [   22.02787399 20734.16796875]
amp, flux bounds, fit_pars: 6 -492.35966912510213 3098.919716155335 [   22.27618389 20967.89453125]
amp, flux bounds, fit_pars: 7 -495.59665861202666 3069.7868107730123 [   22.43797916 21120.1875    ]
amp, flux bounds, fit_pars: 8 -495.27264322688325 3072.7029492393026 [   22.42167804 21104.84375   ]
amp, flux bounds, fit_pars: 9 -485.613252381603 3159.6374668468247 [   21.94636423 20657.4453125 ]
amp, flux bounds, fit_pars: 10 -483.0768761636344 3182.464852808543 [   21.82487772 20543.09375   ]


In [12]:
# Run the PTC task
mask_files = sorted(glob.glob('_'.join((det_name, run, '*mask.fits'))))
%time bot_eo.ptc_task(run, det_name, flat1_files[::2], gains, mask_files=mask_files, flat2_finder=find_flat2)

CPU times: user 1min 10s, sys: 9.2 s, total: 1min 19s
Wall time: 1min 20s


In [13]:
# Run the tearing task
%time bot_eo.tearing_task(run, det_name, flat1_files)

CPU times: user 16.1 s, sys: 859 ms, total: 16.9 s
Wall time: 22.4 s
